In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import pandas as pd

# Load your train and test data
train_df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/DATASCI 266/Final Project/train_set.xlsx")
test_df = pd.read_excel("/content/drive/My Drive/Colab Notebooks/DATASCI 266/Final Project/test_set.xlsx")

In [4]:
!pip install transformers datasets

In [5]:
# Clean missing values
train_df = train_df.dropna(subset=["line1", "line2"])
test_df = test_df.dropna(subset=["line1", "line2"])

In [6]:
from datasets import Dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)

import random

In [7]:
# Define 10 prompt variations
PROMPT_VARIANTS = {
    "P1": "Given this rap line, generate the next line: {line1}",
    "P2": "This is a line in a hip hop song. I want you to generate the next line in the same song: {line1}",
    "P3": "This is a line in a hip hop song. I want you to generate the next line in the same song. Try to match the style from the first line: {line1}",
    "P4": "This is a line in a hip hop song. I want you to generate the next line in the same song. Try to match the style and length from the first line: {line1}",
    "P5": "This is a line in a hip hop song. I want you to generate the next line in the same song. Try to match the general length and style and emotion from the first line. I also want the next line to rhyme: {line1}",
    "P6": "Given this song lyric line, generate the next song lyric line: {line1}",
    "P7": "Take this line from a hip hop song and generate the next line, matching the style, rhyme, and flow: {line1}",
    "P8": "Take this line from a rap song and generate the next line, matching the style, rhyme, and flow: {line1}",
    "P9": "Take this line from a rap song and generate the next line, matching the style, rhyme, and flow. Keep the length and meter consistent as well: {line1}",
    "P10": "Keep the rhyme going. Given current rhyme, write the next rhyme: {line1}",
}

# Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
# Loop through 10 prompt variations
for prompt_id, prompt_template in PROMPT_VARIANTS.items():
    print(f"\n===== Processing prompt variant: {prompt_id} =====")

    # Build prompt + next-line training text
    inputs = []
    for _, row in train_df.iterrows():
        line1 = row["line1"]
        line2 = row["line2"]
        prompt = prompt_template.format(line1=line1.strip())
        full_text = f"{prompt}\n{line2.strip()}"
        inputs.append(full_text)

    # Prepare and split dataset
    dataset = Dataset.from_dict({"text": inputs})
    split_dataset = dataset.train_test_split(test_size=0.1)
    tokenized = split_dataset.map(tokenize, batched=True, remove_columns=["text"])

    train_dataset = tokenized["train"]
    eval_dataset = tokenized["test"]

    # Load model
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    output_path = f"./gpt2-{prompt_id}"
    training_args = TrainingArguments(
        output_dir=output_path,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        logging_dir=f"{output_path}/logs",
        logging_steps=100,
        save_total_limit=2,
        save_steps=500,
        eval_strategy="epoch",
        fp16=True,
        max_steps=5000,
        report_to="wandb",
        run_name=f"gpt2-{prompt_id}"
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()

    # Save model + tokenizer
    trainer.save_model(output_path)
    tokenizer.save_pretrained(output_path)

    print(f"✅ Finished training prompt variant: {prompt_id}")


===== Processing prompt variant: P1 =====


Map:   0%|          | 0/71807 [00:00<?, ? examples/s]

Map:   0%|          | 0/7979 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Map:   0%|          | 0/71807 [00:00<?, ? examples/s]

Map:   0%|          | 0/7979 [00:00<?, ? examples/s]

/tmp/ipython-input-8-1948195629.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
0,2.274400,2.269549


✅ Finished training prompt variant: P10


In [9]:
!pip install rouge_score bert_score evaluate nltk textstat sentence-transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   

In [10]:
import torch
import evaluate
import nltk
import textstat
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer, util
from collections import Counter

nltk.download('punkt')

# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
sbert = SentenceTransformer("all-MiniLM-L6-v2")

# Metric helpers
def calc_bleu(preds, refs):
    return bleu.compute(predictions=preds, references=refs)["bleu"]

def calc_rouge(preds, refs):
    r = rouge.compute(predictions=preds, references=refs)
    return r["rouge1"], r["rouge2"], r["rougeL"]

def calc_bertscore(preds, refs):
    b = bertscore.compute(predictions=preds, references=refs, lang="en")
    return np.mean(b["f1"])

def calc_sbert(preds, refs):
    sims = [util.cos_sim(sbert.encode(p), sbert.encode(r))[0][0].item() for p, r in zip(preds, refs)]
    return np.mean(sims)

def rhyme_score(gen, ref):
    g_last = gen.strip().split()[-1][-2:] if gen.strip() else ""
    r_last = ref.strip().split()[-1][-2:] if ref.strip() else ""
    return int(g_last == r_last)

def syllable_diff(g, r):
    return abs(textstat.syllable_count(g) - textstat.syllable_count(r))

def length_ratio(g, r):
    return len(g.split()) / max(len(r.split()), 1)

def diversity(texts):
    all_words = " ".join(texts).split()
    return len(set(all_words)) / max(len(all_words), 1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
def evaluate_prompt_model(prompt_id, prompt_template, model_path, test_df, max_new_tokens=30):
    # Load model + tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_path).eval().to("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token

    # Store results
    prompts, generated, references = [], [], []

    rhyme_scores, syll_diffs, len_ratios = [], [], []

    for _, row in test_df.iterrows():
        line1 = row["line1"]
        line2 = row["line2"]
        prompt = prompt_template.format(line1=line1.strip())

        # Generate
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
        gen_line = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()

        # Save results
        prompts.append(prompt)
        generated.append(gen_line)
        references.append(line2)
        rhyme_scores.append(rhyme_score(gen_line, line2))
        syll_diffs.append(syllable_diff(gen_line, line2))
        len_ratios.append(length_ratio(gen_line, line2))

    # Global metrics
    bleu_score = calc_bleu(generated, references)
    r1, r2, rL = calc_rouge(generated, references)
    bert_f1 = calc_bertscore(generated, references)
    sbert_sim = calc_sbert(generated, references)
    div = diversity(generated)

    # Print summary
    print(f"\n📊 Prompt: {prompt_id}")
    print(f"BLEU: {bleu_score:.4f}, ROUGE-1: {r1:.4f}, ROUGE-2: {r2:.4f}, ROUGE-L: {rL:.4f}")
    print(f"BERTScore F1: {bert_f1:.4f}, SBERT: {sbert_sim:.4f}, Diversity: {div:.4f}")
    print(f"Avg Rhyme Rate: {np.mean(rhyme_scores):.4f}, Avg Syllable Diff: {np.mean(syll_diffs):.2f}, Length Ratio: {np.mean(len_ratios):.2f}")

    # Optional: return row-by-row results
    import pandas as pd
    df = pd.DataFrame({
        "prompt_id": prompt_id,
        "line1": test_df["line1"].tolist(),  # <-- Add this
        "prompt": prompts,
        "generated_line": generated,
        "reference_line": references,
        "rhyme": rhyme_scores,
        "syllable_diff": syll_diffs,
        "length_ratio": len_ratios,
    })
    return df


In [12]:
test_df_sample = test_df.sample(n=100, random_state=42)

In [13]:
import pandas as pd

# Collect results
all_results = []
for prompt_id, prompt_template in PROMPT_VARIANTS.items():
    model_path = f"./gpt2-{prompt_id}"
    df_result = evaluate_prompt_model(prompt_id, prompt_template, model_path, test_df_sample)
    all_results.append(df_result)

# Merge all results into one DataFrame
final_df = pd.concat(all_results)

# Show result (either head or full)
print(final_df.head(10))  # Show top 10 rows
# OR
from IPython.display import display
display(final_df)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...


📊 Prompt: P5
BLEU: 0.0258, ROUGE-1: 0.0887, ROUGE-2: 0.0347, ROUGE-L: 0.0884
BERTScore F1: 0.8078, SBERT: 0.2431, Diversity: 0.1439
Avg Rhyme Rate: 0.0400, Avg Syllable Diff: 10.93, Length Ratio: 2.83

📊 Prompt: P6
BLEU: 0.0268, ROUGE-1: 0.0909, ROUGE-2: 0.0365, ROUGE-L: 0.0899
BERTScore F1: 0.8166, SBERT: 0.2378, Diversity: 0.1398
Avg Rhyme Rate: 0.0500, Avg Syllable Diff: 10.51, Length Ratio: 2.82



📊 Prompt: P7
BLEU: 0.0269, ROUGE-1: 0.0864, ROUGE-2: 0.0326, ROUGE-L: 0.0860
BERTScore F1: 0.8072, SBERT: 0.2361, Diversity: 0.1429
Avg Rhyme Rate: 0.0700, Avg Syllable Diff: 10.69, Length Ratio: 2.84



📊 Prompt: P8
BLEU: 0.0267, ROUGE-1: 0.0873, ROUGE-2: 0.0331, ROUGE-L: 0.0874
BERTScore F1: 0.8097, SBERT: 0.2385, Diversity: 0.1480
Avg Rhyme Rate: 0.0400, Avg Syllable Diff: 11.69, Length Ratio: 2.97



📊 Prompt: P9
BLEU: 0.0265, ROUGE-1: 0.0843, ROUGE-2: 0.0340, ROUGE-L: 0.0844
BERTScore F1: 0.8078, SBERT: 0.2383, Diversity: 0.1454
Avg Rhyme Rate: 0.0500, Avg Syllable Diff: 11.02, Length Ratio: 2.87

📊 Prompt: P10
BLEU: 0.0263, ROUGE-1: 0.0878, ROUGE-2: 0.0329, ROUGE-L: 0.0878
BERTScore F1: 0.8156, SBERT: 0.2388, Diversity: 0.1312
Avg Rhyme Rate: 0.0500, Avg Syllable Diff: 10.01, Length Ratio: 2.80
  prompt_id                                              line1  \
0        P1               You do it well but hating's (no job)   
1        P1                                 Throw it back, huh   
2        P1                You know that I spent lot of my fit   
3        P1                Canary stones yellow like a pumpkin   
4        P1                           Toric bestii co na pętli   
5        P1                   Leavin' you smashed on tar roads   
6        P1                    Me and my dogs, we bust it down   
7        P1  I just walked up in the court room, I ain't so...   
8

,prompt_id,line1,prompt,generated_line,reference_line,rhyme,syllable_diff,length_ratio
0,P1,You do it well but hating's (no job),"Given this rap line, generate the next line: Y...",You do it well but hating's (no job) You do it...,Nigga for everything I got I had to (work hard),0,12,2.200000
1,P1,"Throw it back, huh","Given this rap line, generate the next line: T...","Throw it back, huh, huh, huh, huh, huh, huh, h...","Throw it back, ooh",0,12,4.000000
2,P1,You know that I spent lot of my fit,"Given this rap line, generate the next line: Y...","time\nI'ma be the one to get you out of here, ...","Got a lot of bad hoes, I just call and place m...",0,8,1.692308
3,P1,Canary stones yellow like a pumpkin,"Given this rap line, generate the next line: C...","I'ma be the one to make it to the top, I'ma be...",Dunkin Donut precious size stones make me wann...,0,4,1.750000
4,P1,Toric bestii co na pętli,"Given this rap line, generate the next line: T...","ę, pętlię\nPętlię, pętlię, pę",Pędzi dźwięk pieniędzy wierz mi,0,0,1.000000
...,...,...,...,...,...,...,...,...
95,P10,"All you do is stop and stare, stop and stare","Keep the rhyme going. Given current rhyme, wri...","All you do is stop and stare, stop and stare, ...","No, you don't love me, yeah",0,17,3.833333
96,P10,But you and I both know the truth'll never die,"Keep the rhyme going. Given current rhyme, wri...","I'ma keep it real, I'ma keep it real, I'ma kee...","Listen, nigga, if you scared, get a dog, but b...",0,0,1.214286
97,P10,"Bought it in the trap, butter knife in the pot","Keep the rhyme going. Given current rhyme, wri...","I'm a real nigga, I'm a real nigga, I'm a real...","I come from the same place, we still hustle wh...",0,8,1.416667
98,P10,"And we could not atone, the money, clothes, an...","Keep the rhyme going. Given current rhyme, wri...","And we could not atone, the money, clothes, an...",And my rep is sell like a boatload,0,14,2.375000
